In [1]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

## Задача - получить rmse на отложенной выборке ниже 0.87

Так как при попытке загрузки всех данных моей оперативной памяти не хватило, пришлось схитрить, поместив обработку данных в функцию для экономии памяти, а также взяв для анализа не весь датасет из 27 миллионнов сток, а всего лишь 10 миллионов (что уже достаточно для  нужного результата)

In [2]:
def get_data():
    movies = pd.read_csv('D:\GitRepo\Нетология\Рекомендательные системы\ml-latest\ml-latest/movies.csv')
    ratings = pd.read_csv('D:\GitRepo\Нетология\Рекомендательные системы\ml-latest\ml-latest/ratings.csv')
    
    movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
    movies_with_ratings.dropna(inplace=True)
    
    print('shape', movies_with_ratings.shape)
    
    dataset = pd.DataFrame({
        'uid': movies_with_ratings.userId,
        'iid': movies_with_ratings.title,
        'rating': movies_with_ratings.rating
    })
    
    print('users:', len(dataset.uid.unique()))
    print('items:',len(dataset.iid.unique()))
    
    r_min = ratings.rating.min()
    r_max = ratings.rating.max()
    
    reader = Reader(rating_scale=(r_min, r_max))
    data = Dataset.load_from_df(dataset.head(10000000), reader)
    
    trainset, testset = train_test_split(data, test_size=.15)
    
    return trainset, testset

In [3]:
trainset, testset = get_data()

shape (27753444, 6)
users: 283228
items: 53817


In [4]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [5]:
test_pred = algo.test(testset)

In [6]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8130


0.8129532692412382

В итоге оценка RMSE модели (0.813) получилась более чем подходящая и существенно меньше, чем 0.87, что требуется для зачета.